# Graph Class

In [1]:
class Graph:
    """
    Graph using an adjacency list with only built-ins.
    - directed: if False, edges are added both ways
    - weights: store as numbers (default 1)
    """
    def __init__(self, directed=False):
        self.directed = directed
        self._adj = {}

    # -------- Vertex ops --------
    def add_vertex(self, v):
        if v not in self._adj:
            self._adj[v] = {}

    def remove_vertex(self, v):
        if v not in self._adj:
            return
        # remove inbound edges:
        for u in list(self._adj.keys()):
            if v in self._adj[u]:
                del self._adj[u][v]
        # remove the vertex itself
        del self._adj[v]

    def has_vertex(self, v):
        return v in self._adj

    # -------- Vertex ops --------
    def add_edge(self, u, v, weight=1):
        self.add_vertex(u)
        self.add_vertex(v)
        self._adj[u][v] = weight
        if not self.directed:
            self._adj[v][u] = weight

    def remove_edge(self, u, v):
        if u in self._adj and v in self._adj[u]:
            del self._adj[u][v]
        if not self.directed and v in self._adj and v in self._adj[u]:
            del self._adj[v][u]

    # -------- Inspection --------
    def get_neighbors(self, v):
        # Return all neighbors of a given vertex v and their edge weights
        return dict(self._adj.get(v, {}))

    def num_vertices(self):
        # Return how many vertices (nodes) exist in the graph
        return len(self._adj)

    def num_edges(self):
        # Return how many edges exist in the graph
        # if the graph is directed: just count all edges.
        # if undirected: divide by 2
        m = sum(len(item) for item in self._adj.values())
        return m if self.directed else m // 2

    def vertices(self):
        # Return a list of all vertices in the graph
        return [key for key in self._adj]

    def edges(self):
        # Return a list of all edges in the form (v, u, weight)
        edges = []
        if self.directed:
            for v in self._adj:
                for u in self._adj[v]:
                    edges.append((v, u, self._adj[v][u]))
        else:
            seen = set()
            for v in self._adj:
                for u in self._adj[v]:
                    if tuple(sorted([v, u])) not in seen:
                        edges.append((v, u, self._adj[v][u]))
                        seen.add(tuple(sorted([v, u])))
        return edges

    def to_adjacency_list(self):
        # Return the graph's adjacency list as a dictionay-of-dictionary
        return {v: dict(nbrs) for v, nbrs in self._adj.items()}

    def to_adjacency_matrix(self):
        # Return a matrix form of the graph
        adj_matrix = [[0] * len(self._adj) for _ in range(len(self._adj))]
        index_dict = {key: index for index, key in enumerate(self._adj)}
        for v in self._adj:
            for u in self._adj[v]:
                adj_matrix[index_dict[v]][index_dict[u]] = self._adj[v][u]
        return adj_matrix

        
    def __str__(self):
        # Define how the graph prints as a string
        lines = [f"{'Directed' if self.directed else 'Undirected'} Graph"]
        for u in self.vertices():
            nbrs = ", ".join(f"{v}({w})" for v, w in self._adj[u].items())
            lines.append(f"   {u}:{nbrs}")
        return "\n".join(lines)

    def print_graph(self) -> None:
        print(self)
        # Just print the string representation of the graph

In [69]:
g = Graph(directed=False)
g.add_edge("A", "B")
g.add_edge("B", "C")
g.add_edge("A", "C")
g.add_edge("C", "D")
g.add_edge("C", "E")
g.print_graph()

Undirected Graph
   A:B(1), C(1)
   B:A(1), C(1)
   C:B(1), A(1), D(1), E(1)
   D:C(1)
   E:C(1)


In [95]:
g = Graph(directed=False)
g.add_edge("A", "B")
g.add_edge("A", "C")
g.add_edge("A", "D")
g.add_edge("C", "E")
g.print_graph()

Undirected Graph
   A:B(1), C(1), D(1)
   B:A(1)
   C:A(1), E(1)
   D:A(1)
   E:C(1)


# DFS Implementation

In [61]:
temp = ['Z', 'A', 'B']
temp.sort()
temp[-1]

'Z'

In [68]:
def dfs_iterative(g: Graph, start: str) -> list:
    result = []
    visited = set()
    stack = [start]
    curr = start
    while len(stack) != 0:
        flag = False
        if curr not in visited:
            result.append(curr)
        ngbrs = sorted(list(g.get_neighbors(curr).keys()))
        visited.add(curr)
        # Get the first item in the neighbor that is not in the visited list
        for node in ngbrs:
            if node not in visited:
                curr = node
                stack.append(curr)
                flag = True
                break
        if not flag:
            stack.pop()
            if len(stack) != 0:
                curr = stack[-1]
    return result


dfs_iterative(g, "A")

['A', 'B', 'C', 'E', 'D']

In [99]:
def dfs_recursive(g: Graph, node: str, visited: list):
    if node in visited:
        return []
    visited.append(node)
    result = [node]
    ngbrs = sorted(list(g.get_neighbors(node).keys()))
    for item in ngbrs:
        if item in visited:
            continue
        else:
            result.extend(dfs_recursive(g, item, visited))
    return result


dfs_recursive(g, "A", [])

['A', 'B', 'C', 'E', 'D']

# Breadth First Search

In [112]:
g = Graph()
g.add_edge('A', 'B')
g.add_edge('A', 'C')
g.add_edge('B', 'C')
g.add_edge('B', 'D')
g.add_edge('C', 'E')
g.add_edge('D', 'E')
print(g)

Undirected Graph
   A:B(1), C(1)
   B:A(1), C(1), D(1)
   C:A(1), B(1), E(1)
   D:B(1), E(1)
   E:C(1), D(1)


In [119]:
g = Graph()
g.add_edge('A', 'B')
g.add_edge('B', 'C')
g.add_edge('A', 'C')
g.add_edge('C', 'D')
g.add_edge('C', 'E')
print(g)

Undirected Graph
   A:B(1), C(1)
   B:A(1), C(1)
   C:B(1), A(1), D(1), E(1)
   D:C(1)
   E:C(1)


In [120]:
def bfs_iterative(g: Graph, start: str) -> list:
    queue = [start]
    visited = set()
    result = []
    num = 0
    while len(queue) != 0:
        num += 1
        node = queue.pop(0)
        if node not in visited:
            visited.add(node)
            result.append(node)
        ngbrs = sorted(list(g.get_neighbors(node).keys()))
        for item in ngbrs:
            if item not in visited:
                queue.append(item)
    return result


bfs_iterative(g, 'A')


Queue: ['A']
Node: A
Visited: set()

Queue: ['B', 'C']
Node: B
Visited: {'A'}

Queue: ['C', 'C']
Node: C
Visited: {'A', 'B'}

Queue: ['C', 'D', 'E']
Node: C
Visited: {'A', 'C', 'B'}

Queue: ['D', 'E', 'D', 'E']
Node: D
Visited: {'A', 'C', 'B'}

Queue: ['E', 'D', 'E']
Node: E
Visited: {'A', 'C', 'D', 'B'}

Queue: ['D', 'E']
Node: D
Visited: {'E', 'B', 'D', 'A', 'C'}

Queue: ['E']
Node: E
Visited: {'E', 'B', 'D', 'A', 'C'}


['A', 'B', 'C', 'D', 'E']

### DFS와 BFS
**Question:**  
그래프를 DFS로 탐색한 결과와 BFS로 탐색한 결과를 출력하는 프로그램을 작성하시오. 단, 방문할 수 있는 정점이 여러 개인 경우에는 정점 번호가 작은 것을 먼저 방문하고, 더 이상 방문할 수 있는 점이 없는 경우 종료한다. 정점 번호는 1번부터 N번까지이다.  
**Input:**  
첫째 줄에 정점의 개수 N(1 ≤ N ≤ 1,000), 간선의 개수 M(1 ≤ M ≤ 10,000), 탐색을 시작할 정점의 번호 V가 주어진다. 다음 M개의 줄에는 간선이 연결하는 두 정점의 번호가 주어진다. 어떤 두 정점 사이에 여러 개의 간선이 있을 수 있다. 입력으로 주어지는 간선은 양방향이다.  
**Output:**  
첫째 줄에 DFS를 수행한 결과를, 그 다음 줄에는 BFS를 수행한 결과를 출력한다. V부터 방문된 점을 순서대로 출력하면 된다.

In [168]:
def dfs(adj_matrix: dict, start:int)->list:
    stack = [start]
    visited = set()
    result = []
    while len(stack) != 0:
        curr = stack.pop()
        if curr in visited:
            continue
        visited.add(curr)
        result.append(curr)
        ngbrs = sorted(adj_matrix[curr], reverse=True)
        for node in ngbrs:
            if node not in visited:
                stack.append(node)
    return result
            
def bfs(adj_matrix: dict, start:int)->list:
    queue = [start]
    visited = set()
    result = []
    head = 0
    while head < len(queue):
        curr = queue[head]
        head += 1
        if curr not in visited:
            visited.add(curr)
            result.append(curr)
        ngbrs = sorted(adj_matrix[curr])
        for node in ngbrs:
            if node not in visited:
                queue.append(node)
    return result
    
    
def dfs_and_bfs():
    V, E, S = map(int, input().split())
    adj_matrix = {i: set() for i in range(1, V+1)}
    for _ in range(E):
        v, u = map(int, input().split())
        if v not in adj_matrix:
            adj_matrix[v] = [u]
        else:
            adj_matrix[v].append(u)
        if u not in adj_matrix:
            adj_matrix[u] = [v]
        else:
            adj_matrix[u].append(v)
    return dfs(adj_matrix, S), bfs(adj_matrix, S)

    
dfs, bfs = dfs_and_bfs()
print(' '.join(map(str, dfs)))
print(' '.join(map(str, bfs)))

 4 5 1
 1 2
 1 3
 1 4
 2 4
 3 4


1 2 4 3
1 2 3 4


### 미로 탐색
**Question:**  
N x M 크기의 배열로 표현되는 미로가 있다.  
미로에서 1은 이동할 수 있는 칸을 나타내고, 0은 이동할 수 없는 칸을 나타낸다. 이러한 미로가 주어졌을 때, (1,1)에 서 출발하여 (N,M)의 위치로 이동할 때 지나야 하는 최소의 칸 수를 구하는 프로그램을 작성하시오. 한 칸에서 다른 칸으로 이동할 때, 서로 인접한 칸으로만 이동할 수 있다.  
**Input:**  
첮째 줄에 두 정수 N, M(2 $\leq$ N, M $\leq$ 100)이 주어진다. 다음 N개의 줄에는 M개의 정수로 미로가 주어진다. 각각의 수들은 붙어서 입력으로 주어진다.  
**Output:**  
첮째 줄에 지나야 하는 최소의 칸 수를 출력한다. 항상 도착위치로 이동할 수 있는 경우만 입력으로 주어진다.

In [12]:
def search_maze():
    from collections import deque
    N, M = map(int, input().split())
    maze = []
    queue = deque([(0,0,1)])
    visited = set()
    for _ in range(N):
        maze.append(list(input()))
    while len(queue) != 0:
        curr = queue.popleft()
        curr_x, curr_y, dist = curr
        if curr in visited:
            continue
        visited.add((curr_x, curr_y))

        if (curr_x, curr_y) == (N-1, M-1):
            print(dist)
            break

        # check up
        if curr_x - 1 >= 0:
            if maze[curr_x - 1][curr_y] == '1' and (curr_x - 1, curr_y) not in visited:
                queue.append((curr_x-1, curr_y, dist+1))
        # check down
        if curr_x + 1 < N:
            if maze[curr_x + 1][curr_y] == '1' and (curr_x + 1, curr_y) not in visited:
                queue.append((curr_x+1, curr_y, dist+1))
        # check left
        if curr_y - 1 >= 0:
            if maze[curr_x][curr_y - 1] == '1' and (curr_x, curr_y - 1) not in visited:
                queue.append((curr_x, curr_y - 1, dist+1))
        # check right
        if curr_y + 1 < M:
            if maze[curr_x][curr_y + 1] == '1' and (curr_x, curr_y + 1) not in visited:
                queue.append((curr_x, curr_y + 1, dist+1))
                
search_maze()

# Version with sys.stdin.readline
# def search_maze():
#     from collections import deque
#     import sys
#     input = sys.stdin.readline   # use fast input

#     N, M = map(int, input().split())
#     maze = []
#     queue = deque([(0,0,1)])
#     visited = set()
#     for _ in range(N):
#         maze.append(list(input().strip()))  # strip() removes newline

#     while queue:
#         curr_x, curr_y, dist = queue.popleft()
#         if (curr_x, curr_y) in visited:
#             continue
#         visited.add((curr_x, curr_y))

#         if (curr_x, curr_y) == (N-1, M-1):
#             print(dist)
#             break

#         # check up
#         if curr_x - 1 >= 0 and maze[curr_x - 1][curr_y] == '1' and (curr_x - 1, curr_y) not in visited:
#             queue.append((curr_x-1, curr_y, dist+1))
#         # check down
#         if curr_x + 1 < N and maze[curr_x + 1][curr_y] == '1' and (curr_x + 1, curr_y) not in visited:
#             queue.append((curr_x+1, curr_y, dist+1))
#         # check left
#         if curr_y - 1 >= 0 and maze[curr_x][curr_y - 1] == '1' and (curr_x, curr_y - 1) not in visited:
#             queue.append((curr_x, curr_y-1, dist+1))
#         # check right
#         if curr_y + 1 < M and maze[curr_x][curr_y + 1] == '1' and (curr_x, curr_y + 1) not in visited:
#             queue.append((curr_x, curr_y+1, dist+1))

# search_maze()

 4 6
 101111
 101010
 101011
 111011


15


### 바이러스
**Question:**  
신종 바이러스인 웜 바이러스는 네트워크를 통해 전파된다. 한 컴퓨터가 웜 바이러스에 걸리면 그 컴퓨터와 네트워트상에서 연결되어 있는 모든 컴퓨터는 윔 바이러스에 걸리게 된다.  
**Input:**  
첮째 줄에는 컴퓨터의 수가 주어진다. 컴퓨터의 수는 100이하인 정수이고 각 컴퓨터에는 1번 부터 차례대로 번호가 매겨진다. 둘째 줄에는 네트워크 상에서 직접 연결되어 있는 컴퓨터 쌍의 수가 주어진다. 이어서 그 수만큼 한 줄에 한 쌍씩 네티워크 상에서 직접 연결되어 있는 컴퓨터의 번호 쌍이 주어진다.  
**Output:**  
1번 컴퓨터가 웜 바이러스에 걸렸을 때, 1번 컴퓨터를 통해 웜 바이러스에 걸리게 되는 컴퓨터의 수를 첮째 줄에 출력한다.

In [25]:
def virus():
    V = int(input())
    E = int(input())
    adj_list = {vertex+1: [] for vertex in range(V)}
    for _ in range(E):
        v, u = map(int, input().split())
        adj_list[v].append(u)
        adj_list[u].append(v)
    # Performing DFS
    visited = {1}
    stack = [1]
    while len(stack) != 0:
        curr = stack.pop()
        ngbrs = adj_list[curr]
        for node in ngbrs:
            if node in visited:
                continue
            stack.append(node)
            visited.add(node)
    print(len(visited) - 1)
virus()

 7
 6
 1 2
 2 3
 1 5
 5 2
 5 6
 4 7


4


### 단지번호붙이기
**Question:**  
<그림 1>과 같이 정사각형 모양의 지도가 있다. 1은 집이 있는 곳을, 0은 집이 없는 곳을 나타낸다. 철수는 이 지도를 가지고 연결된 집의 모임인 단지를 정외하고, 단지에 번호를 붙이려 한다. 여기서 연결되었다는 것은 어떤 집이 좌우 혹은 아래위로 다른 집이 있는 경우를 말한다. 대각선상에 집에 있는 경우는 연결된 것이 아니다. <그림2>는 <그림1>을 단지별로 번호를 붙인 것이다. 지도를 입력하여 단지수를 출력하고, 각 단지에 속하는 집의 수를 오름차순으로 정렬하여 출력하는 프로그램을 작성하시오.  
**Input:**  
첮 번째 줄에는 지도의 크기 N(적사각형이므로 가로와 세로의 크기는 같으며 5$\leq$N$\leq$25)이 입력되고, 그 다음 N줄에 는 각각 N개의 자료(0혹은 1)가 입력된다.  
**Output:**  
첮 번째 줄에는 총 단지수를 출력하시오. 그리고 각 단지내 집의 수를 오름차순으로 정렬하여 한 줄에 하나씩 출력하시오

In [39]:
def bfs(N: int,
        start: tuple[int, int],
        grid:list[list[str]],
        visited:set[tuple[int, int]])->int:
    from collections import deque
    queue = deque([start])
    visited.add(start)
    counter = 1
    while queue:
        curr = queue.popleft()
        x, y = curr
        # search neighbors
        # up
        if x - 1 >= 0:
            if grid[x-1][y] == '1' and (x-1, y) not in visited:
                queue.append((x-1, y))
                visited.add((x-1, y))
                counter += 1
        # down
        if x + 1 < N:
            if grid[x+1][y] == '1' and (x+1, y) not in visited:
                queue.append((x+1, y))
                visited.add((x+1, y))
                counter += 1
        # left
        if y - 1 >= 0:
            if grid[x][y-1] == '1' and (x, y-1) not in visited:
                queue.append((x, y-1))
                visited.add((x, y-1))
                counter += 1
        # right 
        if y + 1 < N:
            if grid[x][y+1] == '1' and (x, y+1) not in visited:
                queue.append((x, y+1))
                visited.add((x, y+1))
                counter += 1
    return counter
        
def count_complex():
    N = int(input())
    grid = []
    visited = set()
    result = []
    for _ in range(N):
        grid.append(list(input()))
    for x in range(N):
        for y in range(N):
            apartment = grid[x][y]
            if apartment == '1' and (x,y) not in visited:
                result.append(bfs(N, (x,y), grid, visited))
    print(len(result))
    for item in sorted(result):
        print(item)
count_complex()

 7
 0110100
 0110101
 1110101
 0000111
 0100000
 0111110
 0111000


3
7
8
9


### 촌수계산  
**Question:**  
우리 나라는 가족 혹은 친척들 사이의 관계를 촌수라는 단위로 표현하는 독특한 문화를 가지고 있다. 이러한 촌수는 다음과 같은 방식으로 계산된다. 기본적으로 부모와 자식 사이를 1촌으로 정의하고 이로부터 사람들 간의 촌수를 계산한다. 예를 들면 나와 아버지, 아버지와 할아버지는 각각 1촌으로 나와 할아버지는 2촌이 되고, 아버지 형제들과 아버지는 1촌, 나의 아버지 형제들과는 3촌이 된다.   여러 사람들에 대한 부모 자식들 간의 관계가 주어졌을 때, 주어진 두 사람의 촌수를 계산하는 프로그램을 작성하시오.  
**Input:**  
사람들은 $1,2,3,...,n (1 \leq n \leq 100) $의 연속된 번호로 각각 표기된다. 입력 파일에 첮째 줄에는 전체 사람의 수 n이 주어지고, 둘째 줄에는 촌수를 계산해야 하는 서로 다른 두 사람의 번호가 주어진다. 그리고 셋째 줄에는 부모 자식들간의 관계의 개수 m이 주어진다. 넷째 줄부터는 부모 자식간의 관계를 나타내는 두 번호 x,y가 각 줄에 나온다. 이때 앞에 나오는 번호 x는 뒤에 나오는 정수 y의 부모 번호를 나타낸다.    
각 사람의 부모는 최대 한 명만 주어진다.  
**Output:**  
입력에서 요구한 두 사람의 촌수를 나타내는 정수를 출력한다. 어떤 경우에는 두 사람의 친척 관계가 전혀 없이 촌수를 계산할 수 없을 때가 있다. 이때에는 -1 를 출력해야 한다.

In [16]:
def kinship_distance():
    from collections import deque
    N = int(input())
    start, end = map(int, input().split())
    M = int(input())
    adj_matrix = {i:[] for i in range(1,N+1)}
    for _ in range(M):
        x, y = map(int, input().split())
        adj_matrix[x].append(y)
        adj_matrix[y].append(x)
    queue = deque([(start, 1)])
    visited = {start}
    while queue:
        curr, dist = queue.popleft()
        ngbrs = adj_matrix[curr]
        for node in ngbrs:
            if node == end:
                return dist
            if node in visited:
                continue
            visited.add(node)
            queue.append((node, dist+1))
    return -1
    

print(kinship_distance())

 9
 8 6
 7
 1 2
 1 3
 2 7
 2 8
 2 9
 4 5
 4 6


-1

### 토마토
*Question:**  
철수의 토마토 농장에서는 토마토를 보관하는 큰 창고를 가지고 있다. 토마토는 아래의 그림과 같이 격자모양 상자의 칸에 하나씩 넣은 다음, 상자들을 수직으로 쌓아 올려서 창고에 보관한다.   
창고에 보관되는 토마토들 중에는 잘 익은 겄도 있지만, 아직 익지 않은 토마토들도 있을 수 있다. 보관 후 하루가 지나면, 익은 토마토를 인접한 곳에 있는 익지 않은 토마토들은 익은 토마토의 영향을 받아 익게 된다. 하나의 토마토에 인접한 곳은 위, 아래, 왼쪽, 오른쪽, 앞, 뒤 여섯 방향에 있는 토마토를 의미한다. 대각선 방향에 있는 토마토들에게는 영향을 주지 못하며, 토마토가 혼자 저졸로 익는 경우는 없다고 가정한다. 철수는 창고에 보관된 토마토들이 며칠이 지나면 다 익게 되는지 그 최소 일수를 알고 싶어 한다.  
토마토를 창고에 보관하는 격자모양의 상자들의 크기와 익은 토마토들과 익지 않은 토마토들의 정보가 주어졌을 때, 며칠이 지나면 토마토들이 모두 익는지, 그 최소 일수를 구하는 프로그램을 작성하라. 단, 상자의 일부 칸에는 토마토가 들어있지 않을 수도 있다.  
**Input:**  
첮 줄에는 상자의 크기를 나타내는 두 정소 M, N과 쌓아올려지는 상자의 수를 나타내는 H가 주어진다. M은 상자의 가로 칸의 수, N은 상자의 세로 칸의 수를 나타낸다. 단, 2$\leq$M$\leq$100, 2$\leq$N$\leq$100, 1$\leq$H$\leq$100 이다. 둘째 줄부터는 가장 밑의 상자부터 가장 위의 상자까지에 저장된 토마토들의 정보가 주어진다. 즉, 둘째 줄부터 N개의 줄에는 하나의 상자에 담긴 토마토의 정보가 주어진다. 각 줄에는 상자 가로줄에 들어있는 토마토들의 상태가 M개의 정수로 주어진다. 정수 1은 익은 토마토, 정수 0은 아직 익지 않은 토마토, 정수 -1은 토마토가 들어있지 않은 칸을 나타낸다. 이러한 N개의 줄이 H번 반족하여 주어진다.  
토마토가 하나 이상 있는 경우만 입력이 주어진다.  
**Output:**  
여러분은 토마토가 모두 익을 때가지 최소 며칠이 걸리는지를 계산해서 출력해야 한다. 만약, 저장될 때부터 모들 토마토가 익어있는 상태이면 0을 출력해야 하고, 토마토가 모두 익지는 못하는 상황이면 -1을 출력해야 한다.

In [81]:
def tomato_two():
    from collections import deque
    M,N,H = map(int, input().split())
    grid = []
    index_ones = []
    all_ripe = True
    # 6 moves in 3D: (dh, dn, dm)
    DIRS3D = [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]
    
    for i in range(H):
        layer = []
        for j in range(N):
            row = list(map(int, input().split()))
            if 0 in row:
                all_ripe = False
            if 1 in row:
                for index, k in enumerate(row):
                    if k  == 1:
                        index_ones.append((i,j,index,0))
            layer.append(row)
        grid.append(layer)
    if all_ripe:
        return 0

    queue = deque(index_ones)
    while queue:
        curr = queue.popleft()
        h,n,m,d = curr
        for dh, dn, dm in DIRS3D:
            nh, nn, nm = h+dh, n+dn, m+dm
            if 0 <= nh < H and 0 <= nn < N and 0 <= nm < M and grid[nh][nn][nm] == 0:
                grid[nh][nn][nm] = 1
                queue.append((nh,nn,nm,d+1))
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if 0 in grid[i][j]:
                return -1
    return d
tomato_two()

 4 3 2
 1 1 1 1
 1 1 1 1
 1 1 1 1
 1 1 1 1
 -1 -1 -1 -1
 1 1 1 -1


0

In [76]:
def tomato():
    from collections import deque
    M,N,H = map(int, input().split())
    grid = []
    index_ones = []
    all_ripe = True
    for i in range(H):
        layer = []
        for j in range(N):
            row = list(map(int, input().split()))
            if 0 in row:
                all_ripe = False
            # if row contains one store the index to the list
            if 1 in row:
                for index, k in enumerate(row):
                    if k == 1:
                        index_ones.append((i,j,index,0))
            layer.append(row)
        grid.append(layer)
    if all_ripe:
        return 0
    queue = deque(index_ones)
    while queue:
        curr = queue.popleft()
        h,n,m,d = curr
        if h-1 >= 0 and grid[h-1][n][m] != -1:
            if grid[h-1][n][m] == 0:
                queue.append((h-1,n,m,d+1))
                grid[h-1][n][m] = 1
        if h+1 < H and grid[h+1][n][m] != -1:
            if grid[h+1][n][m] == 0:
                queue.append((h+1,n,m,d+1))
                grid[h+1][n][m] = 1
        if n-1 >= 0 and grid[h][n-1][m] != -1:
            if grid[h][n-1][m] == 0:
                queue.append((h,n-1,m,d+1))
                grid[h][n-1][m] = 1
        if n+1 < N and grid[h][n+1][m] != -1:
            if grid[h][n+1][m] == 0:
                queue.append((h,n+1,m,d+1))
                grid[h][n+1][m] = 1
        if m-1 >= 0 and grid[h][n][m-1] != -1:
            if grid[h][n][m-1] == 0:
                queue.append((h,n,m-1,d+1))
                grid[h][n][m-1] = 1
        if m+1 < M and grid[h][n][m+1] != -1:
            if grid[h][n][m+1] == 0:
                queue.append((h,n,m+1,d+1))
                grid[h][n][m+1] = 1
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if 0 in grid[i][j]:
                return -1
    return d
print(tomato())

 4 3 2
 1 1 1 1
 1 1 1 1
 1 1 1 1
 1 1 1 1
 -1 -1 -1 -1
 1 1 1 -1


0

### 숨바꼭질  
**Question:**  
수빈이는 동생과 숨바꼭질을 하고 있다. 수빈이는 현재 점 N(0$\leq$N$\leq$100,000)에 있고, 동생의 점 K(0$\leq$K$\leq$100,000)에 있다. 수빈이는 걷거나 순간이동을 할수 있다. 만약, 수빈이의 위치가 X일 때 걷는다면 1초 후에 X-1 또는 X+1로 이동하게 된다. 순간이동을 하는 경우에는 1초 후에 2*X의 위치로 이동하게 된다.  
수빈이와 동생의 위치가 주어졌을 때, 수빈이가 동생을 찾을 수 있는 가장 빠른 시간이 몇 초 후인지 구하는 프로그램을 작성하시오.  
**Input:**  
첮 번쨰 줄에 수빈이가 있는 위치 N과 동생이 있는 위치 K가 주어진다. N과 K는 정수이다.  
**Output:**  
수빈이가 동생을 찾는 가장 빠른 시간을 출력한다.  

In [103]:
def hide_and_seek():
    from collections import deque
    N, M = map(int, input().split())
    if N == M:
        return 0
    queue = deque([(N,0)])
    visited = {N}
    while queue:
        curr, time = queue.popleft()
        ngbrs = [2*curr, curr-1, curr+1]
        for node in ngbrs:
            if node in visited:
                continue
            if node == M:
                return time + 1
            queue.append((node, time+1))
            visited.add(node)
    return time

print(hide_and_seek())

 0 1


1

In [3]:
def hide_and_seek():
    from collections import deque

    MAX = 100000
    N, M = map(int, input().split())
    if N == M:
        return 0

    queue = deque([(N, 0)])
    visited = [False] * (MAX + 1)
    visited[N] = True

    while queue:
        curr, time = queue.popleft()
        for node in (curr * 2, curr - 1, curr + 1):
            if 0 <= node <= MAX and not visited[node]:   # clamp to bounds
                if node == M:
                    return time + 1
                visited[node] = True
                queue.append((node, time + 1))

    return -1  # unreachable (shouldn't happen for this problem)

print(hide_and_seek())

 5 17


4


### 스트링크  
**Question:**  
강호는 코딩 교육을 하는 스타트업 스타링크에 지원했다. 오늘은 강호의 면접날이다. 하지만, 늦잠을 잔 강호는 스타링크가 있는 건물에 늦게 도착하고 말았다.  
스트링크는 총 F층으로 이루어진 고층 건물에 사무실에 있고, 스타링트가 있는 곳의 위치는 G층이다. 강호가 지금 있는 곳은 S층이고, 이제 엘리베이터를 타고 G층으로 이동하려고 한다.  
보통 엘리베이터에는 어떤 층으로 이동할 수 있는 버튼이 있지만, 강호가 탄 엘리베이터는 버튼이 2개밖에 없다. U버튼은 위로 U층을 가는 버튼, D버튼은 아래로 D층을 가는 버튼이다. (만약, U층 위, 또는 D층 아래에 해당하는 층이 없을 때는, 엘리베이터는 움직이지 않는다)  
강호가 G층에 도착하려먼, 버튼을 적어도 몇 번 눌러야 하는지 프로그램을 작성하시오. 만약, 엘리베이터를 이용해서 G층에 갈 수 없으면, "use the stairs"를 출력한다.  
**Input:**  
첮째 줄에 F,S,G,U,D가 주어진다. (1$\leq$S, G$\leq$F$\leq$1000000, 0$\leq$U, D$\leq$1000000) 건물은 1층부터 시작하고, 가장 높은 층은 F층이다.  
**Output:**  
첮째 줄에 강호가 S층에서 G층으로 가기 위해 눌러야 하는 버튼의 수의 최솟값을 출력한다. 만약, 엘리베이터로 이동 할 수 없을 때는 "use the stairs"를 출력한다.

In [19]:
def starlink():
    from collections import deque
    F,S,G,U,D = map(int, input().split())
    if S == G:
        return 0
    queue = deque([(S, 0)])
    visited = {S}
    iteration = 0
    while queue:
        curr = queue.popleft()
        floor, count = curr
        ngbrs = [floor+U, floor-D]
        iteration += 1
        for node in ngbrs:
            # check if it is out of bound
            if 1 <= node <= F and node not in visited:
                queue.append((node, count + 1))
                visited.add(node)
                if node == G:
                    return count + 1
    return "use the stairs"

print(starlink())

 10 1 10 2 1


6


### 맥주 마시면서 걸어가기  
**Question:**  
송도에 사는 상근이와 친구들은 송도에서 열리는 펜타포트 락 페스티벌에 가려고 한다. 올해는 맥주를 마시면서 걸어 가기로 했다. 출발은 상근이네 집에서 하고, 맥주 한 박스를 들고 출발한다. 맥주 한 박스에는 맥주가 20개 들어있다. 목이 마르면 안되기 때문에 50미터에 한 병씩 마시려고 한다. 즉, 50미터를 가려면 그 직전에 맥주 한병을 마셔야 한다.  
상근이의 집에서 페스티벌이 열리는 곳은 매우 먼 거리이다. 따라서, 맥주를 더 구매해야 할 수도 있다. 미리 인터넷으로 조사를 해보니 다행이도 맥주를 파는 편의점이 있다. 편의점에 들렸을 때, 빈 병은 버리고 새 맥주 병을 살 수 있다. 하지만, 박스에 들어있는 맥주는 20병을 넘을 수 없다. 편의점을 나선 직후에도 50미터를 가기 전에 맥주 한 병을 마셔야 한다.  
편의점, 상근이네 집, 펜타포트 락 페스티벌의 좌표가 주어진다. 상근이와 친구들이 행복하게 페스티벌에 도착할 수 있는지 구하는 프로그램을 작성하시오.  
**Input:**  
첯째 줄에 테스트 케이스와 개수 t가 주어진다. (t$\leq$50)  
각 테스트 케이스의 첮재 줄에는 맥주를 파는 편의점의 개수 n이 주어진다. (0$\leq$n$\leq$).
다음 n+2개 줄에는 상근이네 집, 편의점, 펜타포트 락 페스티벌 좌표가 주어진다. 각 좌표는 두 정수 x와 y로 이루어져 있다. (두 값 모두 미터. -32768$\leq$x,y$\leq$32767)  
송도는 직사각형 모양으로 생긴 도시이다. 두 좌표 사이의 거리는 x 좌표의 차이 +y 죄표의 차이 이다. (맨해튼 거리)  
**Output:**  
각 테스트 케이스에 대해서 상근이와 친구들이 행복하게 페스티벌에 갈 수 있어면 "happy", 중간에 맥주가 바닥나서 더 이동할 수 없으면 "sad"를 출력한다. 

In [35]:
def beer_walk():
    test_case = int(input())
    results = []
    for _ in range(test_case):
        result = 'sad'
        adj_list = {}
        num_store = int(input())
        start = tuple(map(int, input().split()))
        adj_list[start] = []
        for _ in range(num_store):
            store = tuple(map(int, input().split()))
            adj_list[store] = []
            for node in adj_list:
                if node == store:
                    continue
                if abs(node[0] - store[0]) + abs(node[1] - store[1]) <= 1000:
                    adj_list[store].append(node)
                    adj_list[node].append(store)
        festival = tuple(map(int, input().split()))
        adj_list[festival] = []
        for node in adj_list:
            if node == festival:
                continue
            if abs(node[0] - festival[0]) + abs(node[1] - festival[1]) <= 1000:
                adj_list[node].append(festival)
                adj_list[festival].append(node)


        # Run BFS
        from collections import deque
        queue = deque([start])
        visited = {start}
        while queue:
            curr = queue.popleft()
            ngbrs = adj_list[curr]
            for node in ngbrs:
                if node in visited:
                    continue
                if node == festival:
                    result = 'happy'
                queue.append(node)
                visited.add(node)
        results.append(result)
    for result in results:
        print(result)


beer_walk()

 2
 2
 0 0
 1000 0
 1000 1000
 2000 1000
 2
 0 0
 1000 0
 2000 1000
 2000 2000


happy
sad


### 로봇 청소기
**Question:**  
로봇 청소기와 방의 상태가 주어졌을 때, 청소하는 영역의 개수를 구하는 프로그램을 작성하시오.

로봇 청소기가 있는 방은 N x M 크기의 직사각형으로 나타낼수 있으며, 1 x 1 크기의 정사각형 칸으로 나누어져 있다. 각각의 칸은 벽 또는 빈 칸이다. 청소기는 바라보는 방향이 있으며, 이 방향은 동, 서, 남, 북 중 하나이다. 방의 각 칸은 좌표 (r,c)로 나타낼 수 있고, 가장 북쪽 중의 가장 서쪽 칸의 좌표가 (0,0), 가장 남쪽 줄의 가장 동쪽 칸의 좌표가 (N-1, M-1)이다. 즉, 좌표 (r,c)는 북쪽에서 (r+1)번째에 있는 줄의 서쪽에서 (c+1)번째 칸을 가르킨다. 처음에 빈 칸은 전부 청소되지 않은 상태이다.

로봇 청소기는 다음과 같이 작동한다.

1. 현재 칸이 아직 청소되지 않은 경우, 현재 칸을 청소한다.
2. 현재 칸의 주면 4칸중 청소되지 않은 빈 칸이 없는 경우,
     1. 바라보는 방향을 유지한 채로 한 칸 후진할 수 있다면 한 칸 후진하고 1번으로 돌아간다.
     2. 바라보는 방향의 뒤쪽 칸이 벽이라 후진할 수 없다면 작동을 멈춘다.
3. 현재 칸의 주변 4칸ㄴ 중 청소되지 않은 빈 칸이 있는 경우,
     1. 반시계 방향으로 90° 회전한다.
     2. 바라보는 방향을 기준으로 앞쪽 칸이 청소되지 않은 빈 칸인 경우 한 칸 전진한다.
     3. 1번으로 돌아간다.


**Input:**  
첮째 줄에 방의 크기 N과 M이 입력된다. (3 $\leq$ N, M $\leq$ 50) 둘째 줄에 처음에 로봇 청소기가 있는 칸의 좌표 (r, c)와 처음에 로봇 청소기가 바라보는 방향 d가 입력된다. d가 0인 경우 북쪽, 1인 경우 동쪽, 2인 경우 남쪽, 3인 경우 서쪽을 바라보고 있는 것이다.

셋째 줄버터 N개의 줄에 각 장소의 상태를 나타내는 N x M개의 값이 한 줄에 M개씩 입력된다. i번째 줄의 j번 째 깞은 칸 (i,j)의 상태를 나타내며, 이 값은 0인 경우 (i,j)가 청소되지 않은 빈 칸이고, 1인 경우 (i,j)에 벽이 있는 것이다. 방의 가장 북쪽, 가장 남쪽, 가장 서쪽, 가장 동쪽 줄 중 하나 이상에 위치한 모든 칸에는 벽이 있다. 로봇 청소기가 있는 칸은 항상 빈 칸이다. 


**Output:**  
로봇 청소기가 작동을 시작한 후 작동을 멈출 때까지 청소하는 칸의 개수를 출력한다.

In [83]:
def robot():
    N, M = map(int, input().split())
    r, c, d = map(int, input().split())
    grid = []
    for _ in range(N):
        grid.append(list(map(int, input().split())))
    print(grid)
robot()

KeyboardInterrupt: Interrupted by user

In [87]:
N, M = 3, 4
r, c, d = 1, 1, 1
grid = [[1, 0, 1, 1], [1, 0, 0, 1], [1, 1, 1, 1]]
grid

[[1, 0, 1, 1], [1, 0, 0, 1], [1, 1, 1, 1]]

In [86]:
# if d is 0:North(Top), 1:East(Right), 2:South(Down), 3:West(Left)
# 1. if the current space is not cleaned, clean it
# 2. if the four sides of the current space are cleaned:
#    2.1 face the same direction and if possible move back
#    2.2 If it is not possible to move back, stop the machine
# 3. If there is a side (from the four) that is not currently cleaned
#    3.1 Move the machine 90° anti-clock wise (left)
#    3.2 If the side that is facing is not cleaned move forward and clean it
#    3.3 Go back to 1

def check_surrounding(grid, x, y, N, M):
    """
    return True if there is surrounding that can be cleaned else return False
    """
    return True if grid[x+1][y] == 0 or grid[x-1][y] == 0 or grid[x][y+1] == 0 or grid[x][y-1] == 0 else False


count = 0
x, y = r, c
iteration = 0
while True:
    print(f'iteration: {iteration}')
    iteration += 1
    if grid[x][y] == 0:
        count += 1
        grid[x][y] = 2
    # check the four sides of the current position
    if not check_surrounding(grid, x, y, N, M):  # if there is nothing to clean
        if d == 0:  # facing north
            if grid[x+1][y] != 1:
                x = x+1
            else:
                break
        if d == 1:  # facing east
            if grid[x][y-1] != 1:
                y = y-1
            else:
                break
        if d == 2:  # facing south
            if grid[x-1][y] != 1:
                x = x-1
            else:
                break
        if d == 4:  # facing west
            if grid[x][y+1] != 1:
                y = y + 1
            else:
                break
    else:  # if there is a place to clean
        # move the direction
        if d-1 < 0:
            d = 3
        else:
            d -= 1
        if d == 0:  # facking north
            if grid[x-1][y] != 1:
                x = x-1
        if d == 1:  # facing east
            if grid[x][y+1] != 1:
                y = y+1
        if d == 2:  # facing south
            if grid[x+1][y] != 1:
                x = x+1
        if d == 3:  # facking west
            if grid[x][y-1] != 1:
                y = y - 1
    print(count)

iteration: 0
1
iteration: 1
2
iteration: 2
2
iteration: 3
2
iteration: 4
2
iteration: 5
3
iteration: 6


### 안전 영역


**Question:**  
재난방재청에서는 많은 비가 내리는 장마철에 대비해서 다음과 같은 일을 계확하고 있다. 먼저 어떤 지역의 높이 정보를 파악한다. 그 다음에 그 지역에 많은 비가 내렸을 때 물에 잠기지 않은 안전한 영역이 최대로 몇 개가 만들어 지는 지를 조사하려고 한다. 이때, 문제를 간단하게 하기 위하여 장마철에 내리는 비의 양에 따라 일정한 높이 이하의 모든 지점은 물에 잠긴다고 가정한다.

어쩐 지역의 높은 정보는 행과 열의 크기가 각각 N인 2차원 배열 형태로 주어지며 배열의 각 원소는 해당 지점의 높이를 표시하는 지수이다. 예를 들어 다음은 N=5인 지역의 높이 정보이다.

<table border="1" cellspacing="0" cellpadding="8" style="border-collapse: collapse; text-align: center;">
  <tr>
    <td>6</td>
    <td>8</td>
    <td>2</td>
    <td>6</td>
    <td>2</td>
  </tr>
  <tr>
    <td>3</td>
    <td>2</td>
    <td>3</td>
    <td>4</td>
    <td>6</td>
  </tr>
  <tr>
    <td>6</td>
    <td>7</td>
    <td>3</td>
    <td>3</td>
    <td>2</td>
  </tr>
  <tr>
    <td>7</td>
    <td>2</td>
    <td>5</td>
    <td>3</td>
    <td>6</td>
  </tr>
  <tr>
    <td>8</td>
    <td>9</td>
    <td>5</td>
    <td>2</td>
    <td>7</td>
  </tr>
</table>


이제 위와 같은 지경에 많은 비가 내려서 높이가 4 이하인 모든 지점이 물에 잠겼다고 하자. 이 경우에 물에 잠기는 지점을 회색으로 표시하면 다음과 같다.

<table>
  <tr>
    <td>6</td>
    <td>8</td>
    <td style="background-color:lightgray;">2</td>
    <td>6</td>
    <td style="background-color:lightgray;">2</td>
  </tr>
  <tr>
    <td style="background-color:lightgray;">3</td>
    <td style="background-color:lightgray;">2</td>
    <td style="background-color:lightgray;">3</td>
    <td>4</td>
    <td style="background-color:lightgray;">6</td>
  </tr>
  <tr>
    <td>6</td>
    <td>7</td>
    <td style="background-color:lightgray;">3</td>
    <td style="background-color:lightgray;">3</td>
    <td style="background-color:lightgray;">2</td>
  </tr>
  <tr>
    <td>7</td>
    <td style="background-color:lightgray;">2</td>
    <td>5</td>
    <td>3</td>
    <td>6</td>
  </tr>
  <tr>
    <td>8</td>
    <td>9</td>
    <td>5</td>
    <td style="background-color:lightgray;">2</td>
    <td>7</td>
  </tr>
</table>


물에 잠기지 않은 안전한 영역이라 함은 물에 잠기자 않은 지점들이 위, 아래, 오른쪽 혹은 왼쪽으로 인접해 있으며 그 크기가 최대인 영역을 말한다. 위의 경우에서 물에 잠기지 않은 안전한 영역은 5개가 된다(꼭짓점으로만 붙어 있는 두 지점은 인접하지 않는다고 취급한다).

또한 위와 같은 지역에서 높이가 6이하인 지점을 모두 잠기게 만드는 많은 비가 내리면 물에 잠기지 않은 안전한 영역은 아래 그림에서와 같이 네 개가 됨을 확인할 수 있다.

<table border="1" cellspacing="0" cellpadding="8" style="border-collapse: collapse; text-align: center;">
  <tr>
    <td style="background-color:lightgray;">6</td>
    <td>8</td>
    <td style="background-color:lightgray;">2</td>
    <td style="background-color:lightgray;">6</td>
    <td style="background-color:lightgray;">2</td>
  </tr>
  <tr>
    <td>3</td>
    <td style="background-color:lightgray;">2</td>
    <td style="background-color:lightgray;">3</td>
    <td style="background-color:lightgray;">4</td>
    <td style="background-color:lightgray;">6</td>
  </tr>
  <tr>
    <td style="background-color:lightgray;">6</td>
    <td>7</td>
    <td style="background-color:lightgray;">3</td>
    <td style="background-color:lightgray;">3</td>
    <td style="background-color:lightgray;">2</td>
  </tr>
  <tr>
    <td>7</td>
    <td style="background-color:lightgray;">2</td>
    <td style="background-color:lightgray;">5</td>
    <td style="background-color:lightgray;">3</td>
    <td style="background-color:lightgray;">6</td>
  </tr>
  <tr>
    <td>8</td>
    <td>9</td>
    <td style="background-color:lightgray;">5</td>
    <td style="background-color:lightgray;">2</td>
    <td>7</td>
  </tr>
</table>

이와 같이 장마철에 내리는 비의 양에 따라서 물에 잠기지 않은 안전한 영역의 개수는 다르게 된다. 위와 예가 같은 지역에서 내리는 비의 양에 따른 모든 경우를 다 조사해 보면 물에 잠기지 않은 안전한 영역의 개수 중에서 최대인 경우는 5임을 알 수 있다.

어떤 지역의 높이 정보가 주어졌을 때, 장마청에 물에 잠기지 않은 안전한 영역의 최대 개수를 계산하는 프로그램을 작성하시오.

**Input:**  
첮째 줄에는 어떤 지역을 나타내는 2차원 배열의 행과 열의 개수를 나타내는 수 N이 입력된다. N은 2 이상 100 이하의 정수이다. 둘째 줄부터 N개의 각 줄에는 2차원 배열의 첫 번째 행부터 N번째 행까지 순서대로 한 행씩 높이 정보가 입력된다. 각 줄에는 각 행의 첫 번째 열부터 N번째 열까지 N개의 높이 정보를 나타내는 자연수가 빈 칸을 사이에 두고 입력된다. 높이는 1이상 100 이하의 정수이다.

**Output:**  
첮째 줄에 장마철에 물이 잠기지 않은 안전한 영역의 최대 개수를 출력한다.

In [101]:
# 1. go through the grid and whenever you find area above the rain height
# 2. Run the DFS (have a global visited set), add the all the node visited
#    to the global visited
# 3. whenver you run a DFS increment the counter
# 4. when you loop through the whole grid return the counter
def safe_area():
    # taking in the input
    N = int(input())
    grid = []
    for _ in range(N):
        grid.append(list(map(int, input().split())))

    max_h = max(map(max, grid))
    max_counter = 0    

    # helper function
    def safe_area_dfs(N, h, index, visited, grid):
        stack = [index]
        visited.add(index)
        while stack:
            curr = stack.pop(-1)
            x, y = curr
            # get the neighbors of current node
            # up 
            if x-1 >= 0 and grid[x-1][y] > h and (x-1, y) not in visited:
                visited.add((x-1, y))
                stack.append((x-1, y))
            # down
            if x+1 < N and grid[x+1][y] > h and (x+1, y) not in visited:
                visited.add((x+1, y))
                stack.append((x+1, y))
            # left 
            if y-1 >= 0 and grid[x][y-1] > h and (x, y-1) not in visited:
                visited.add((x, y-1))
                stack.append((x, y-1))
            # right 
            if y+1 < N and grid[x][y+1] > h and (x, y+1) not in visited:
                visited.add((x, y+1))
                stack.append((x, y+1))
                
    # actual algorithm
    for h in range(0, max_h+1):
        visited = set()
        counter = 0
        for row in range(N):
            for column in range(len(grid[row])):
                if (row, column) not in visited and grid[row][column] > h:
                    safe_area_dfs(N, h, (row, column), visited, grid)
                    counter += 1
        max_counter = max(max_counter, counter)
    return max_counter 

    
print(safe_area())

 3
 1 1 1
 1 1 1
 1 1 1


1


### 빙산
**Question:**  
지구 온난화로 인하여 북극의 빙산이 녹고 있다. 빙산을 그림 1과 같이 2차원 배열에 표시한다고 하자. 빙산의 각 부분별 높이 정보는 배열의 각 칸에 양의 정수로 지정된다. 빙산 이외의 바다에 해당되는 칸에는 0이 지정된다. 그림 1에서 빈칸 모두 0으로 채워져 있다고 생각한다.

![그림 1. 행의 개수가 5이고 열의 개수가 7인 2차원 배열에 저장된 빙산의 높이 정보](images/iceberg_1.png)

빙산의 높이는 바닷물에 많이 접해있는 부분에서 더 빨리 줄어들기 때문에, 배열에서 빙산의 각 부분에 해당되는 칸에 있는 높이는 일년마다 그 칸에 동서남북 네 방향으로 붙어있는 0이 지정된 칸의 개수만큼 줄어든다. 단, 각 칸에 지정된 높이는 0보다 더 줄어들지 않는다. 바닷물은 호수처럼 빙산에 둘러싸여 있을 수도 있다. 따라서 그림 1의 빙산은 일년후에 그림 2와 같이 변형된다.

그림 3은 그림1읜 방산이 2년 후에 변한 모습을 보여준다. 2차원 배열에서 동서남북 방향으로 붙어있는 칸들은 서로 연결되어 있다고 말한다. 따라서 그림 2의 방산은 한 덩어리지만, 그림 3의 빙산은 세 덩어리를 분리되어 있다.

![그림 2](images/iceberg_2.png)
![그림 3](images/iceberg_3.png)

한 덩어리의 빙산이 주어질 때, 이 빙산이 두 덩어리 이상으로 분리되는 최초의 시간(년)을 구하는 프로그램을 작성하시오. 그림 1의 빙산에 대해서는 2가 답이다. 만일 전부 녹을 때까지 두 덩어리 이상으로 분리되지 않으면 프로그램은 0을 출력한다.


**Input:**  
첮 줄에는 이차원 배열의 행의 개수와 열의 개수를 나타내는 두 정수 N과 M이 한 개의 빈칸을 사이에 두고 주어진다. N과 M은 3 이상 300이하이다. 그 다음 N개의 줄에는 각 줄마다 배열의 각 행을 나타내는 M개의 정수가 한개의 빈 칸을 사이에 두고 주어진다. 각 칸에 들어가는 값은 0이상 10이하이다. 배열에서 빙산이 차지하는 칸의 개수, 즉, 1 이상의 정수가 들어가는 칸의 개수는 10,000개 이하이다. 배열의 첫 번째 행과 열, 마지막 행과 열에는 항상 0으로 채워진다.


**Output:**  
첫 줄에 빙산이 분리되는 최초의 시간(년)을 출력한다. 만일 빙산이 다 녹을 때까지 분리되지 않으면 0을 출력한다.

In [146]:
def ice_berg():
    N, M = map(int, input().split())
    grid = []
    year = 0
    for i in range(N):
        grid.append(list(map(int, input().split())))
    def ice_berg_bfs(N,M,index,grid,visited):
        from collections import deque
        queue = deque([index])
        visited.add(index)
        while queue:
            curr = queue.popleft()
            x, y = curr
            # check the neighbors of current iceberg
            if x-1 >= 0 and grid[x-1][y] != 0 and (x-1, y) not in visited: #  up
                queue.append((x-1, y))
                visited.add((x-1,y))
            if x+1 < N and grid[x+1][y] != 0 and (x+1, y) not in visited: #  down
                queue.append((x+1, y))
                visited.add((x+1,y))
            if y-1 >= 0 and grid[x][y-1] != 0 and (x, y-1) not in visited: #  left
                queue.append((x, y-1))
                visited.add((x, y-1))
            if y+1 < M and grid[x][y+1] != 0 and (x, y+1) not in visited: #  right
                queue.append((x, y+1))
                visited.add((x, y+1))
    previous_year = [row[:] for row in grid]
    while True:
        this_year = [row[:] for row in previous_year]
        # Run bfs
        counter = 0
        visited = set()
        for row in range(N):
            for column in range(M):
                if this_year[row][column] != 0 and (row, column) not in visited:
                    ice_berg_bfs(N, M, (row, column), this_year, visited)
                    counter += 1
        if counter > 1:
            return year
        # Re-adjust the icebergs
        for row in range(N):
            for column in range(M):
                if previous_year[row][column] > 0: #  if the item is not water
                    # Check the surrounding
                    if row-1 >= 0 and previous_year[row-1][column] == 0: #  Up and if it is water
                        if this_year[row][column] - 1 < 0: #  if it is below 0, just set it to 0
                            this_year[row][column] = 0
                        else:
                            this_year[row][column] -= 1
                    if row+1 < N and previous_year[row+1][column] == 0: #  Down and if it is water
                        if this_year[row][column] - 1 < 0: #  
                            this_year[row][column] = 0
                        else:
                            this_year[row][column] -= 1
                    if column-1 >= 0 and previous_year[row][column-1] == 0: #  Left and if it is water
                        if this_year[row][column] - 1 < 0:
                            this_year[row][column] = 0
                        else:
                            this_year[row][column] -= 1
                    if column+1 < M and previous_year[row][column+1] == 0: #  Right and if it is water
                        if this_year[row][column] - 1 < 0:
                            this_year[row][column] = 0
                        else:
                            this_year[row][column] -= 1
        year += 1
        # Check if it is all melted
        flag = True
        for row in range(N):
            for column in range(M):
                if this_year[row][column] != 0:
                    flag = False
        if flag:
            return 0
        previous_year = [row[:] for row in this_year]
                    
print(ice_berg())

 5 7
 0 0 0 0 0 0 0
 0 2 4 0 4 2 0
 0 3 5 0 5 3 0
 0 0 0 0 0 0 0
 0 0 0 0 0 0 0


0